In [2]:
from openai import OpenAI
from random import randint, choice
from os import environ
from pathlib import Path
from json import loads, dumps
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_3_5_turbo_completion, answerConversion = lambda x: x, maxTries = 10):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}. Last answer: {answer}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [1]:
categories = ["particles", "experiments", "units", "constants", "equations", "theories", "scientists", "instruments", "materials", "substances", "phenomena", "effects", "laws", "technologies", "discoveries", "inventions", "hypotheses", "theorems", "paradoxes", "problems", "conjectures", "proofs", "algorithms"]

In [7]:
from pathlib import Path
datapath = Path("../master-database-files/master-experimental/ontology_test/")

In [3]:
def categoryExamplesInPhysics(category, number = 5):
    query = f'Give me {number} examples of {category} in physics. Return them in a list formatted as ["example1", "example2", ...]. Return nothing but the list.'
    answer, success = tryRecieveAnswer(query, answerConversion = listAnswerConversion)
    if success:
        return answer

In [8]:
def saveCategoryExamplesInPhysics(categories, number = 5):
    examplespercategory = {}
    for category in categories:
        examples = categoryExamplesInPhysics(category, number)
        if examples:
            examplespercategory[category] = examples
    with (datapath / "examplespercategory.json").open("w") as f:
        f.write(dumps(examplespercategory, indent = 4))

In [9]:
saveCategoryExamplesInPhysics(categories, 5)

In [3]:
physicsTerms = ["electron", "solar system", "planc's constant", "quantum mechanics", "relativity", "black hole", "big bang", "quantum field theory", "string theory", "dark matter", "dark energy", "quantum entanglement"] 

In [14]:
def categorySplittingPhysicsTerms(terms):
    termsString = "[" + ', '.join([f'"{term}"' for term in terms]) + "]"
    query = f'''
The separation into the categories "scientist" and "no scientists" splitts the terms ["atom", "einstein", "light", "maxwell"] into two equal sized groups. 
"einstein" and "maxwell" belong to the "scientist" category. "atom" and "light" belong to the "no scientists" category.
What categories splitts the terms {termsString} into two equal sized groups of terms that belong to the category and terms that do not belong to the category?
Return nothing but the two categories formatted as ["category1", "category2"].
'''
    answer, success = tryRecieveAnswer(query, answerConversion = listAnswerConversion)
    if success:
        return answer

In [13]:
categorySplittingPhysicsTerms(physicsTerms)

['theoretical physics', 'observational astronomy']

In [18]:
gpt_3_5_turbo_completion("Is water an object? Answer with Y or N.")

'N'

In [47]:
from owlready2 import *

In [59]:
onto = get_ontology("http://quantsimulant.de/ontology/test.owl")

In [60]:
class TestClass(Thing):
    namespace = onto

In [61]:
class TestSubClass(TestClass):
    namespace = onto

In [62]:
class TestObjectProperty(TestClass >> TestClass):
    namespace = onto

In [63]:
tc = TestClass("tc")

In [64]:
onto.save(file = str(datapath / "test.owl"), format = "ntriples")

In [65]:
import rdflib
r = [*default_world.sparql_query("SELECT ?s ?pr ?o WHERE { ?s ?pr ?o .}")]
r

[[.anonymous, 6, owl.Ontology],
 [ontology.test.owl, 6, owl.Ontology],
 [test.TestClass, 6, 11],
 [test.TestClass, 9, owl.Thing],
 [test.TestSubClass, 6, 11],
 [test.TestSubClass, 9, test.TestClass],
 [test.TestObjectProperty, 6, owl.ObjectProperty],
 [test.TestObjectProperty, 7, test.TestClass],
 [test.TestObjectProperty, 8, test.TestClass],
 [test.tc, 6, owl.NamedIndividual],
 [test.tc, 6, test.TestClass]]

In [66]:
graph = default_world.as_rdflib_graph()

In [68]:
[*graph.query("SELECT ?s ?pr ?o WHERE { ?s ?pr ?o .}")]

[(rdflib.term.URIRef('http://anonymous'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology')),
 (rdflib.term.URIRef('http://quantsimulant.de/ontology/test.owl'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology')),
 (rdflib.term.URIRef('http://quantsimulant.de/ontology/test.owl#TestClass'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('http://quantsimulant.de/ontology/test.owl#TestClass'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Thing')),
 (rdflib.term.URIRef('http://quantsimulant.de/ontology/test.owl#TestSubClass'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.or